In [64]:
import pandas as pd
# 读取JSON文件
df_reagent_prediction = pd.read_json('reagent_prediction.json')
df_retrosynthesis = pd.read_json('retrosynthesis.json')
df_forward_reaction_prediction = pd.read_json('forward_reaction_prediction.json')

In [30]:
import pandas as pd
import numpy as np

#dataframes = [df_reagent_prediction, df_retrosynthesis, df_reactions]
dataframes = [df_reactions]
for df in dataframes:

    mask = df['metadata'].apply(lambda x: x.get('split') == 'train')
    df_train = df[mask]

    n_samples = min(len(df_train), 1000)
    df_sampled = df_train.sample(n=n_samples, random_state=42) 


    df_sampled['metadata'] = df_sampled['metadata'].apply(lambda x: {**x, 'split': 'valid'})


    df.update(df_sampled)

    df.loc[mask, 'metadata'] = df.loc[mask, 'metadata'].apply(lambda x: {**x, 'split': 'train' if x.get('split') != 'valid' else 'valid'})

In [32]:
file_names = ['reagent_prediction.json', 'retrosynthesis.json', 'forward_reaction_prediction.json']

dfs = [df_reagent_prediction, df_retrosynthesis, df_reactions]

for df, file_name in zip(dfs, file_names):
    df.to_json(file_name, orient='records')

In [67]:
df_combined = pd.concat([df_reagent_prediction, df_retrosynthesis, df_forward_reaction_prediction], ignore_index=True)

In [2]:
df_reagent_prediction.head()

,instruction,input,output,metadata
0,"Based on the given chemical reaction, can you ...",[O][=C][C][=C][C][Branch1][=Branch1][N+1][=Bra...,[C][C][C][O][C][Ring1][Branch1].[Cl].[BH4-1].[...,"{'task': 'reagent prediction', 'split': 'train'}"
1,Can you provide potential reagents for the fol...,[C][O][C][=Branch1][C][=O][C][C][C][C][C][C][C...,[C][O].[O].[Na+1].[OH1-1],"{'task': 'reagent prediction', 'split': 'train'}"
2,Please suggest some possible reagents that cou...,[C][O][C][=Branch1][C][=O][C][=C][C][=C][Branc...,[C][C][C][O][C][Ring1][Branch1].[O].[Li+1].[OH...,"{'task': 'reagent prediction', 'split': 'train'}"
3,"Given this chemical reaction, what are some re...",[C][C][=Branch1][C][=O][O][C][Branch1][C][C][=...,[C][C][N][Branch1][Ring1][C][C][C][C].[Cl][C][...,"{'task': 'reagent prediction', 'split': 'train'}"
4,Can you suggest some reagents that might have ...,[C][=C][C][Br].[C][C][O][C][=C][C][=C][Branch1...,[C][C][Branch1][C][C][=O].[O][=C][Branch1][C][...,"{'task': 'reagent prediction', 'split': 'train'}"


In [3]:
len(df_reagent_prediction)

125384

In [15]:
len(df_combined)

380452

In [5]:
len(df_retrosynthesis)

129684

In [16]:
print(df_combined.head())

                                         instruction  \
0  Based on the given chemical reaction, can you ...   
1  Can you provide potential reagents for the fol...   
2  Please suggest some possible reagents that cou...   
3  Given this chemical reaction, what are some re...   
4  Can you suggest some reagents that might have ...   

                                               input  \
0  [O][=C][C][=C][C][Branch1][=Branch1][N+1][=Bra...   
1  [C][O][C][=Branch1][C][=O][C][C][C][C][C][C][C...   
2  [C][O][C][=Branch1][C][=O][C][=C][C][=C][Branc...   
3  [C][C][=Branch1][C][=O][O][C][Branch1][C][C][=...   
4  [C][=C][C][Br].[C][C][O][C][=C][C][=C][Branch1...   

                                              output  \
0  [C][C][C][O][C][Ring1][Branch1].[Cl].[BH4-1].[...   
1                          [C][O].[O].[Na+1].[OH1-1]   
2  [C][C][C][O][C][Ring1][Branch1].[O].[Li+1].[OH...   
3  [C][C][N][Branch1][Ring1][C][C][C][C].[Cl][C][...   
4  [C][C][Branch1][C][C][=O].[O][=C][Branch1][

In [70]:
df_combined.to_json('reactions.json', orient='records')

In [6]:
import pandas as pd
# 读取JSON文件
df_reagent_prediction = pd.read_json('reagent_prediction.json')
df_retrosynthesis = pd.read_json('retrosynthesis.json')
df_forward_reaction_prediction = pd.read_json('forward_reaction_prediction.json')
df_reactions = pd.read_json('reactions.json')

In [9]:
import re
from rdkit import Chem
import selfies as sf
from tqdm.auto import tqdm
import pandas as pd
import os
from rdkit.Chem import MolToSmiles


def selfies_to_smiles(selfie):
    smiles = sf.decoder(selfie) 
    mol = Chem.MolFromSmiles(smiles) 
    if mol:
        canonical_smiles = MolToSmiles(mol, isomericSmiles=True)  # 生成规范化的SMILES
        return canonical_smiles
    return None  


def split_keeping_delimiters(s):
    return re.split(r'(\.|\>\>)', s)


def preprocess_and_filter_dataframe(df):
    drop_indexes = []  

    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
 
        input_parts = split_keeping_delimiters(row['input'])
        input_smiles = []
        for part in input_parts:
            if part in ['.', '>>']:
                input_smiles.append(part)
            else:
                smiles = selfies_to_smiles(part)
                if smiles:
                    input_smiles.append(smiles)
                else:
                    drop_indexes.append(index)
                    break 


        if index not in drop_indexes:
            output_parts = split_keeping_delimiters(row['output'])
            output_smiles = []
            for part in output_parts:
                if part in ['.', '>>']:
                    output_smiles.append(part)
                else:
                    smiles = selfies_to_smiles(part)
                    if smiles:
                        output_smiles.append(smiles)
                    else:
                        drop_indexes.append(index)
                        break  


        if index not in drop_indexes:
            df.at[index, 'input'] = ''.join(input_smiles)
            df.at[index, 'output'] = ''.join(output_smiles)


    df.drop(index=drop_indexes, inplace=True)
    return df

def convert_and_save(df, filename, output_dir='SMILES'):
    os.makedirs(output_dir, exist_ok=True)  
    processed_df = preprocess_and_filter_dataframe(df) 
    output_path = os.path.join(output_dir, filename)  
    processed_df.to_json(output_path, orient='records')  
    print(f"Saved {filename} to {output_path}")


dataframes = {
    'reagent_prediction.json': df_reagent_prediction,
    'retrosynthesis.json': df_retrosynthesis,
    'forward_reaction_prediction.json': df_forward_reaction_prediction,
    'reactions.json': df_reactions
}

for filename, df in dataframes.items():
    convert_and_save(df, filename)


Processing rows:   0%|          | 0/125384 [00:00<?, ?it/s]

[09:22:47] WARNING: not removing hydrogen atom without neighbors
[09:22:47] WARNING: not removing hydrogen atom without neighbors
[09:22:47] WARNING: not removing hydrogen atom without neighbors
[09:22:47] WARNING: not removing hydrogen atom without neighbors
[09:22:47] WARNING: not removing hydrogen atom without neighbors
[09:22:47] WARNING: not removing hydrogen atom without neighbors
[09:22:47] WARNING: not removing hydrogen atom without neighbors
[09:22:47] WARNING: not removing hydrogen atom without neighbors
[09:22:47] WARNING: not removing hydrogen atom without neighbors
[09:22:47] WARNING: not removing hydrogen atom without neighbors
[09:22:47] WARNING: not removing hydrogen atom without neighbors
[09:22:48] WARNING: not removing hydrogen atom without neighbors
[09:22:48] WARNING: not removing hydrogen atom without neighbors
[09:22:48] WARNING: not removing hydrogen atom without neighbors
[09:22:48] WARNING: not removing hydrogen atom without neighbors
[09:22:48] WARNING: not r

Saved reagent_prediction.json to SMILES/reagent_prediction.json


Processing rows:   0%|          | 0/129684 [00:00<?, ?it/s]

Saved retrosynthesis.json to SMILES/retrosynthesis.json


...

[09:30:32] WARNING: not removing hydrogen atom without neighbors
[09:30:33] WARNING: not removing hydrogen atom without neighbors
[09:30:33] WARNING: not removing hydrogen atom without neighbors
[09:30:33] WARNING: not removing hydrogen atom without neighbors
[09:30:33] WARNING: not removing hydrogen atom without neighbors
[09:30:33] WARNING: not removing hydrogen atom without neighbors
[09:30:33] WARNING: not removing hydrogen atom without neighbors
[09:30:33] WARNING: not removing hydrogen atom without neighbors
[09:30:33] WARNING: not removing hydrogen atom without neighbors
[09:30:33] WARNING: not removing hydrogen atom without neighbors
[09:30:33] WARNING: not removing hydrogen atom without neighbors
[09:30:33] WARNING: not removing hydrogen atom without neighbors
[09:30:33] WARNING: not removing hydrogen atom without neighbors
[09:30:33] WARNING: not removing hydrogen atom without neighbors
[09:30:33] WARNING: not removing hydrogen atom without neighbors
[09:30:33] WARNING: not r

Saved forward_reaction_prediction.json to SMILES/forward_reaction_prediction.json


Processing rows:   0%|          | 0/380452 [00:00<?, ?it/s]

[09:34:50] WARNING: not removing hydrogen atom without neighbors
[09:34:50] WARNING: not removing hydrogen atom without neighbors
[09:34:50] WARNING: not removing hydrogen atom without neighbors
[09:34:50] WARNING: not removing hydrogen atom without neighbors
[09:34:50] WARNING: not removing hydrogen atom without neighbors
[09:34:50] WARNING: not removing hydrogen atom without neighbors
[09:34:50] WARNING: not removing hydrogen atom without neighbors
[09:34:51] WARNING: not removing hydrogen atom without neighbors
[09:34:51] WARNING: not removing hydrogen atom without neighbors
[09:34:51] WARNING: not removing hydrogen atom without neighbors
[09:34:51] WARNING: not removing hydrogen atom without neighbors
[09:34:51] WARNING: not removing hydrogen atom without neighbors
[09:34:51] WARNING: not removing hydrogen atom without neighbors
[09:34:51] WARNING: not removing hydrogen atom without neighbors
[09:34:51] WARNING: not removing hydrogen atom without neighbors
[09:34:51] WARNING: not r

Saved reactions.json to SMILES/reactions.json
